In [1]:
import numpy as np
import pandas as pd
import datetime

In [7]:
valid_travel_link_df = pd.read_csv('../data/preprocess/valid_travel_link.csv', parse_dates=['date', 'twenty_minutes_date', 'start_time'])

In [8]:
link_df = pd.read_csv('../data/original/links (table 3).csv', index_col=0)

# 时间区间 ：7月19日-10月24日
每个ink输出对应的两个文件 分别是上午的4个小时和下午的4个小时，计算每前2小时的1/4、1/2、3/4位后两小时的到达时间均值, 测试集数据只有2小时

In [21]:
-

100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123


In [26]:
for link_id in link_df.index.values:
    #print(link_id)
    for m in range(2):
        #print(m)
        time_df = pd.DataFrame(link_time_dict[link_id][m]).T
        time_df = time_df.fillna(0)
#         temp_time_df = time_df.reset_index(drop=True)

#         for i in range(91, 98):
#             for t in range(7):
#                 if t == 6:
#                     temp_time_df.iloc[i][t] = 0
                    
#                 elif np.isnan(temp_time_df.loc[i][t]) :
#                     print('nan: id=%d m=%d i=%d t=%d'%(link_id, m, i, t))

#         time_df[np.array(range(7))] = temp_time_df[np.array(range(7))].values

        if m == 0:
            time_df.to_csv('../data/process/link_median_time/morning/' + str(link_id) + '.csv')

        else:
            time_df.to_csv('../data/process/link_median_time/afternoon/' + str(link_id) + '.csv')


# 计算预测区间内每20分钟的平均流量数据，用于验证

In [27]:
am_start_date = pd.to_datetime('2016-09-11')
am_end_date = pd.to_datetime('2016-10-24')
link_time_dict = {}

for link_id in link_df.index.values:
    print(link_id)
    appoint_link_df = valid_travel_link_df[valid_travel_link_df['link'] == link_id]
    morning_link_time_dict = {}
    afternoon_link_time_dict = {}
    
    norm_date = am_start_date
    while (norm_date <= am_end_date ):
        morning_time_start = norm_date + datetime.timedelta(hours=8)
        morning_time_end = norm_date + datetime.timedelta(hours=10)
        
        afternoon_time_start = norm_date + datetime.timedelta(hours=17)
        afternoon_time_end = norm_date + datetime.timedelta(hours=19)
        
        day_morning_means = []
        prev_time = current_time = morning_time_start
        for p in range(6):
            current_time = current_time + datetime.timedelta(minutes=20)
            temp_df = appoint_link_df[(appoint_link_df['start_time'] >= prev_time) & (appoint_link_df['start_time'] < current_time)]
            if len(temp_df) == 0:
                day_morning_means.append(np.nan)
            else:
                day_morning_means.append(temp_df['consume'].mean())
            
            prev_time = current_time
            
        ############################################################################
        day_afternoon_means = []
        prev_time = current_time = afternoon_time_start
        for p in range(6):
            current_time = current_time + datetime.timedelta(minutes=20)
            temp_df = appoint_link_df[(appoint_link_df['start_time'] >= prev_time) & (appoint_link_df['start_time'] < current_time)]
            if len(temp_df) == 0:
                day_afternoon_means.append(np.nan)
            else:
                day_afternoon_means.append(temp_df['consume'].mean())
            
            prev_time = current_time
        
        ############################################################################

        morning_link_time_dict[norm_date] = day_morning_means
        afternoon_link_time_dict[norm_date] = day_afternoon_means
       
        norm_date = norm_date + datetime.timedelta(days=1)
        
    link_time_dict[link_id] = [morning_link_time_dict, afternoon_link_time_dict]

100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123


In [28]:
#前三天 + 后三天的均值，取到多少算多少，遇到NAN 则跳过
def getMeanTime(df, i, t, min_index, max_index):
    s = 0
    c = 0
    for k in range(1, 5):
        if (i - k) >= min_index :
            s = s + df.loc[i-k][t]
            c = c + 1
            
        if ((i + k) <= max_index) and ~np.isnan(df.loc[i + k][t]):
            s = s + df.loc[i+k][t]
            c = c + 1
    return s / c

In [32]:
for link_id in link_df.index.values:
    for m in range(2):
        time_df = pd.DataFrame(link_time_dict[link_id][m]).T
        temp_time_df = time_df.reset_index(drop=True)
        for i in range(len(temp_time_df)):
            for t in range(6):
                if np.isnan(temp_time_df.loc[i][t]) :
                    temp_time_df.iloc[i][t] = getMeanTime(temp_time_df, i, t, 0, len(temp_time_df) - 1)
        
        time_df[np.array(range(6))] = temp_time_df[np.array(range(6))].values
        
        if m == 0:
            time_df.to_csv('../data/process/link_median_time/morning/minute_' + str(link_id) + '.csv')
        else:
            time_df.to_csv('../data/process/link_median_time/afternoon/minute_' + str(link_id) + '.csv')
            

RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
RangeIndex(start=0, stop=44, step=1)
44
